In [1]:
# importing important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score
# import packages for hyperparameters tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [2]:
data=pd.read_csv(r"D:\Python Home Work\ML programming\Wholesale customers data.csv")

In [3]:
data

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


In [5]:
X = data.drop('Channel' , axis =1 )
y = data['Channel']

In [6]:
X

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,3,12669,9656,7561,214,2674,1338
1,3,7057,9810,9568,1762,3293,1776
2,3,6353,8808,7684,2405,3516,7844
3,3,13265,1196,4221,6404,507,1788
4,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...
435,3,29703,12051,16027,13135,182,2204
436,3,39228,1431,764,4510,93,2346
437,3,14531,15488,30243,437,14841,1867
438,3,10290,1981,2232,1038,168,2125


In [7]:
X.head()

,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,3,12669,9656,7561,214,2674,1338
1,3,7057,9810,9568,1762,3293,1776
2,3,6353,8808,7684,2405,3516,7844
3,3,13265,1196,4221,6404,507,1788
4,3,22615,5410,7198,3915,1777,5185


In [8]:
y.head()

0    2
1    2
2    2
3    1
4    2
Name: Channel, dtype: int64

In [9]:
# • We can see that the y label contain values as 1 and 2.
• We will need to convert it into 0 and 1 for further analysis.

SyntaxError: invalid character '•' (U+2022) (3400009864.py, line 2)

In [10]:
# convert labels into binary values
y[y == 2] = 0
y[y == 1] = 1

In [11]:
y.head()

0    0
1    0
2    0
3    1
4    0
Name: Channel, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train , X_test, y_train , y_test = train_test_split(X , y , test_size = 0.3, random_state = 0)

In [ ]:
#1 Bayesian Optimization with HYPEROPT
Bayesian optimization is optimization or finding the best parameter for a machine learning or deep
learning algorithm.
Optimization is the process of finding a minimum of cost function , that determines an overall
better performance of a model on both train-set and test-set.
In this process, we train the model with various possible range of parameters until a best fit model
is obtained.
Hyperparameter tuning helps in determining the optimal tuned parameters and return the best fit
model, which is the best practice to follow while building an ML or DL model.
In this section, we discuss one of the most accurate and successful hyperparameter tuning method,
which is Bayesian Optimization with HYPEROPT.

Please see my kernel Bayesian Optimization using HYPEROPT, for more information on the op-
timization process using HYPEROPT.

In [ ]:
#2 Bayesian Optimization implementation
Initialize domain space for range of values

In [14]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),

'gamma': hp.uniform ('gamma', 1,9),
'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
'reg_lambda' : hp.uniform('reg_lambda', 0,1),
'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
'n_estimators': 180,
'seed': 0
}

In [ ]:
The available hyperopt optimization algorithms are -
hp.choice(label, options) — Returns one of the options, which should be a list or tuple.
hp.randint(label, upper) — Returns a random integer between the range [0, upper).
hp.uniform(label, low, high) — Returns a value uniformly between low and high.
hp.quniform(label, low, high, q) — Returns a value round(uniform(low, high) / q) * q, i.e it rounds
the decimal values and returns an integer.
hp.normal(label, mean, std) — Returns a real value that’s normally-distributed with mean and
standard deviation sigma.

In [ ]:
# 3 Define objective function

In [18]:
def objective(space):
    clf=xgb.XGBClassifier(

    n_estimators =space['n_estimators'], max_depth =int(space['max_depth']), gamma = space['gamma'],

    reg_alpha =int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
    colsample_bytree=int(space['colsample_bytree']))

    evaluation = [( X_train, y_train), ( X_test, y_test)]
    clf.fit(X_train, y_train,
    eval_set=evaluation, eval_metric="auc",
    early_stopping_rounds=10,verbose=False)

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
#Optimization algorithm

In [19]:
trials = Trials()
best_hyperparams = fmin(fn = objective,
space = space,
algo = tpe.suggest,
max_evals = 100,
trials = trials)

  0%|                                                                          | 0/100 [00:00<?, ?trial/s, best loss=?]

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(



SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
  2%|▉                                              | 2/100 [00:01<00:53,  1.84trial/s, best loss: -0.6515151515151515]

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  

SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
  5%|██▎                                            | 5/100 [00:01<00:19,  4.97trial/s, best loss: -0.6515151515151515]

...

SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
 97%|████████████████████████████████████████████▌ | 97/100 [00:13<00:00,  7.66trial/s, best loss: -0.6515151515151515]

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  

SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
SCORE:                                                                                                                 
0.6515151515151515                                                                                                     
SCORE:                                                                                                                 
 99%|█████████████████████████████████████████████▌| 99/100 [00:13<00:00,  7.28trial/s, best loss: -0.6515151515151515]

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(

C:\Users\SABITA\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  

0.6515151515151515                                                                                                     
100%|█████████████████████████████████████████████| 100/100 [00:13<00:00,  7.21trial/s, best loss: -0.6515151515151515]
